In [22]:
import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv")
base = ['engine_displacement','horsepower','vehicle_weight','model_year','fuel_efficiency_mpg']
df = df[base]

In [43]:
#Question 1

df.isnull().sum()

engine_displacement      0
horsepower             708
vehicle_weight           0
model_year               0
fuel_efficiency_mpg      0
dtype: int64

In [44]:
#Question 2

df['horsepower'].median()

np.float64(149.0)

In [51]:
#Question 3

base = ['horsepower']

np.random.seed(42)

n =len(df)
n_val = int(n*0.2)
n_test = int(n*0.2)
n_train = n - (n_val + n_test)

idx = np.arange(n)
np.random.shuffle(idx)

df_shuffled = df.iloc[idx]
df_train = df_shuffled.iloc[idx[:n_train]].copy()
df_val = df_shuffled.iloc[idx[n_train:n_train+n_val]].copy()
df_test = df_shuffled.iloc[idx[n_train+n_val:]].copy()

y_train = np.log1p(df_train.fuel_efficiency_mpg.values)
y_val = np.log1p(df_val.fuel_efficiency_mpg.values)
y_test = np.log1p(df_test.fuel_efficiency_mpg.values)

In [27]:
base = ['horsepower']

In [46]:
def prepare_X0(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

def prepare_Xm(df):
    df_num = df[base]
    df_num = df_num.fillna(df_train[base].mean())
    X = df_num.values
    return X


In [47]:
def rmse(y,y_pred):
    se = (y-y_pred)**2
    mse = se.mean()
    return np.sqrt(mse)

In [48]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [52]:
X_val = prepare_X0(df_val)
w_0, w = train_linear_regression(X_val, y_val)

y_pred = w_0 + X_val.dot(w)
rmse(y_pred,y_val)

np.float64(0.16330280869823766)

In [53]:
X_val = prepare_Xm(df_val)
w_0, w = train_linear_regression(X_val, y_val)

y_pred = w_0 + X_val.dot(w)
rmse(y_pred,y_val)

np.float64(0.1623737560933444)

In [34]:
#rsme(df_m,df_val['horsepower'])

In [54]:
#Question 4

def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

In [55]:
for r in [0, 0.01, 0.1, 1, 5, 10,100]:
    X_val = prepare_X(df_val)
    w_0, w = train_linear_regression_reg(X_val, y_val, r=r)

    y_pred = w_0 + X_val.dot(w)
    print(r,rmse(y_val,y_pred))

0 0.16330280869823766
0.01 0.16330281434946411
0.1 0.16330337332455663
1 0.16335877003800905
5 0.16464382841843817
10 0.16836313683227114
100 0.33183216883360095


In [58]:
#Question 5

seeds = range(10)
base = ['horsepower']
rmses = []

for seed in seeds:
    np.random.seed(seed)
    
    n =len(df)
    n_val = int(n*0.2)
    n_test = int(n*0.2)
    n_train = n - (n_val + n_test)
    
    idx = np.arange(n)
    np.random.shuffle(idx)
    
    df_shuffled = df.iloc[idx]
    df_train = df_shuffled.iloc[idx[:n_train]].copy()
    df_val = df_shuffled.iloc[idx[n_train:n_train+n_val]].copy()
    df_test = df_shuffled.iloc[idx[n_train+n_val:]].copy()
    
    y_train = np.log1p(df_train.fuel_efficiency_mpg.values)
    y_val = np.log1p(df_val.fuel_efficiency_mpg.values)
    y_test = np.log1p(df_test.fuel_efficiency_mpg.values)

    X_val = prepare_X(df_val)
    w_0, w = train_linear_regression(X_val, y_val)

    y_pred = w_0 + X_val.dot(w)
    rmses.append(rmse(y_val,y_pred))

print(round(np.std(rmses),3))

0.002


In [ ]:
#Question 6
